In [1]:
import os

In [2]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/layeghmorteza/Holland_Barret.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="layeghmorteza"
os.environ["MLFLOW_TRACKING_PASSWORD"]="47ea8e5519d9fbb79e294bf09a018544a39aa22d"


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/layeghmorteza/Holland_Barret.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
#from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def evaluate_clf(self,true, predicted):
        '''
        This function takes in true values and predicted values
        Returns: Accuracy, F1-Score, Precision, Recall, Roc-auc Score
        '''
        acc = accuracy_score(true, predicted) # Calculate Accuracy
        f1 = f1_score(true, predicted) # Calculate F1-score
        precision = precision_score(true, predicted) # Calculate Precision
        recall = recall_score(true, predicted)  # Calculate Recall
        roc_auc = roc_auc_score(true, predicted) #Calculate Roc
        return acc, f1 , precision, recall, roc_auc

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # test_x = test_data.drop([self.config.target_column], axis=1)
        # test_y = test_data[[self.config.target_column]]
        
        X_test = test_data.iloc[:, :-1]
        y_test = test_data.iloc[:, -1]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)

            (acc, f1, precision,recall,roc_auc) = self.evaluate_clf(y_test, predicted_qualities)
            
            # Saving metrics as local
            scores = {"acc": acc, "f1": f1, "precision": precision, "recall": recall, "roc_auc": roc_auc}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("acc", acc)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("roc_auc", roc_auc)
        

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBoost")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-23 14:56:42,941: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-23 14:56:42,943: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-23 14:56:42,946: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-23 14:56:42,947: INFO: common: created directory at: artifacts]
[2024-02-23 14:56:42,948: INFO: common: created directory at: artifacts/model_evaluation]
[2024-02-23 14:56:44,944: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\layeg\anaconda3\envs\mlproj\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'XGBoost' already exists. Creating a new version of this model...
2024/02/23 14:56:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBoost, version 7
Created version '7' of model 'XGBoost'.


In [ ]:
set MLFLOW_TRACKING_URI=https://dagshub.com/layeghmorteza/Holland_Barret.mlflow

set MLFLOW_TRACKING_USERNAME=layeghmorteza 

set MLFLOW_TRACKING_PASSWORD=67b443677363044e3f8fb7b3d6b09887d744602f